<h1><center>Prediction of YouTube Video Popularity</center></h1>
<center></center>
<ol> 
    <center>Orfeas Bourchas PhD StudentSchool of Naval Architecture & Marine Engineering NTUA</center>
    <center>Nikos Silionis PhD StudentSchool of Naval Architecture & Marine Engineering NTUA</center>
    <center>Dimitris Tsoumpelis PhD StudentSchool of Naval Architecture & Marine Engineering NTUA</center>
<ol>


<h2><center> Model Creating and Training for the Quantitative Variables</center></h2>

<h2><center>Importing Libraries </center></h2>

In [ ]:
# Core Data Analysis Libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt 

# Scikit Learn Modules
from sklearn.model_selection import train_test_split, StratifiedShuffleSplit
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import  MinMaxScaler
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC 
from sklearn.metrics import f1_score,  accuracy_score, classification_report, ConfusionMatrixDisplay

# Tensorflow Library
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input

# Optuna Library for parameter optimization
import optuna 

# Set seed_value to be able to reproduce the results
seed_value = 42

<h2><center> Load the Dataset</center></h2>

In [ ]:
os.getcwd()
os.chdir('D:\PhD_Material\PhD_MSc\Machine_Learning\ML_Homeworks\Homework_2\Data')

df = pd.read_csv('Model_Dataset.csv')



<h2><center> Create Variable Combinations </center></h2>

In [ ]:
list_1 = ['views', 'likes', 'dislikes', 'comment_count','days_till_trending','number_of_tags','title_number_of_trending_times', 'class']
list_2 = ['views', 'days_till_trending', 'number_of_tags',    'title_number_of_trending_times', 'title_views_dif', 'class']
list_3 = ['views', 'likes', 'class']
list_4 = ['views', 'likes', 'dislikes', 'comment_count',
       'days_till_trending', 'number_of_tags',
       'title_number_of_trending_times', 'video_id_number_of_trending_times',
       'title_dislikedf', 'video_id_likedf', 'video_id_dislikedf',
       'title_tagsdif', 'video_id_tagsdif', 'title_time_dif',
       'title_views_dif', 'video_id_time_dif', 'video_id_views_dif',
       'title_comment_count_dif', 'video_id_comment_count_dif',
       'comments_to_views', 'likes_to_dislikes', 'class']
input_list = [list_1, list_2, list_3, list_4]

<h2><center> Create Dataset Creation function </center></h2>

In [ ]:
def dataset_creation(df,columns):
    df1 = df[columns].copy()
    split = StratifiedShuffleSplit(n_splits = 1, test_size = 0.3, random_state = seed_value)
    for train_index,test_index in split.split(df1,df1["class"]):
        train_set_strat = df1.copy().loc[train_index]
        test_set_strat = df1.copy().loc[test_index]
    train_set = train_set_strat.copy()
    test_set =test_set_strat.copy()
    train_set_data = train_set.copy().drop(columns = 'class')
    train_set_labels = train_set.copy()["class"]
    test_set_data = test_set.copy().drop(columns = 'class')
    test_set_labels = test_set.copy()["class"]
    scaler = MinMaxScaler()
    pipe = Pipeline([('scale', scaler)])

    train_set_data_tr = pipe.fit_transform(train_set_data)
    test_set_data_tr = pipe.transform(test_set_data)
    train_set_labels_np = train_set_labels.to_numpy()
    test_set_labels_np = test_set_labels.to_numpy()
    return train_set_data_tr, test_set_data_tr, train_set_labels_np, test_set_labels_np, pipe

<h2><center> Train Gaussian Naive Bays classifier </center></h2>

In [ ]:
response_accuracy = {}

for i, name in enumerate(input_list):
    train_set_data_tr, test_set_data_tr, train_set_labels_np, test_set_labels_np, pipe = dataset_creation(df,name)
    
    # Initialize the gnb classifier object.
    gnb = GaussianNB()
    
    # Fit the training data to the classifier
    model = gnb.fit(train_set_data_tr, train_set_labels_np)
    y_pred = model.predict(test_set_data_tr)
    print(classification_report(test_set_labels_np, y_pred))
    
    # Estimate the accuracy.
    response_accuracy[f'gaussia naive bayes_{i+1}'] = round(gnb.score(test_set_data_tr, test_set_labels_np),3)
    
    disp = ConfusionMatrixDisplay.from_predictions(test_set_labels_np, y_pred)
    disp.figure_.suptitle(f"GNB Confusion Matrix for the {i+1} combination")
    plt.savefig(f"GNB Confusion Matrix for the {i+1} combination.jpg")

<h1><center> Optuna Optimization </center></h1>

<h2><center> SVC Objective Function </center></h2>

In [ ]:
def objective_SVC(trial):
    
    model = SVC()
    C = trial.suggest_loguniform('C', 1e-10, 1)
    kernel = trial.suggest_categorical('kernel',['poly','rbf','sigmoid'])
    degree = trial.suggest_int('degree',3, 8)
    params = {
        'C': C,
        'kernel':kernel,
        'degree': degree,
    }
    model.set_params(**params)
    model.fit(train_set_data_tr, train_set_labels_np)

    accuracy = model.score(test_set_data_tr, test_set_labels_np) 
    print("Train Score:",model.score(test_set_data_tr, test_set_labels_np))
    print("\n=================")

    return accuracy

<h2><center> Running SVC Optimization </center></h2>

In [ ]:
for i, name in enumerate(input_list):
    study_name = f"studies//SVC_study_{i}"  
    storage_name = f"sqlite:///{study_name}.db"
    train_set_data_tr, test_set_data_tr, train_set_labels_np, test_set_labels_np, pipe = dataset_creation(df,name)
    study = optuna.create_study(directions=["maximize"],study_name=study_name, storage=storage_name)
    study.optimize(objective_SVC, n_trials=10)

<h2><center> MPL Model Creation and Objective Function </center></h2>

In [ ]:
def create_classifier(trial):
    # We optimize the numbers of layers and their units.
    num_layers = trial.suggest_int("num_layers", low = 1, high = 4, step = 1)
    model = tf.keras.Sequential()
    model.add(Input(train_set_data_tr.shape[1]))
    model.add(Dense(5*train_set_data_tr.shape[1],activation='relu',kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0)))
    for i in range(num_layers):
        neurons = trial.suggest_int(f"neurons_{i}", low = 120, high = 240, step = 120)
        model.add(Dense(neurons, activation="relu",kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0)))
    model.add(Dense(80,activation='relu',kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0)))
    model.add(Dense(4, activation='softmax'))
    return model
def create_optimizer(trial):
    kwargs = {}
    kwargs["learning_rate"] = trial.suggest_float("adam_learning_rate", 1e-5, 1e-1, log=True)
    optimizer = getattr(tf.optimizers, "Adam")(**kwargs)
    return optimizer

In [ ]:
def objective_MLP(trial):
    
    model = create_classifier(trial)
    optimizer = create_optimizer(trial)
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])
    epochs = trial.suggest_int("epochs", low = 100, high = 200, step = 100)
    batch_size = trial.suggest_int("batch_size", low = 10, high = 50, step = 10)
    model.fit( tf.convert_to_tensor(train_set_data_tr), train_set_labels_np, epochs=epochs, batch_size=batch_size)
    accuracy = model.evaluate(test_set_data_tr, test_set_labels_np)[1]
    return accuracy

<h2><center> Running MPL Optimization </center></h2>

In [ ]:
for i, name in enumerate(input_list):
    study_name = f"MLP_study_{i}"  # Unique identifier of the study.
    storage_name = f"sqlite:///{study_name}.db"
    train_set_data_tr, test_set_data_tr, train_set_labels_np, test_set_labels_np, pipe = dataset_creation(df,name)
    study = optuna.create_study(directions=["maximize"],study_name=study_name, storage=storage_name)
    study.optimize(objective_MLP, n_trials=50)

<h1><center> Recreating Best models </center></h1>

<h2><center>SVC</center></h2>

In [ ]:
for i, name in enumerate(input_list):
    comb = i+1
    # Loading the study
    study_name = f"studies//SVC_study_{i}"  
    storage_name = f"sqlite:///{study_name}.db"
    loaded_study = optuna.load_study(study_name=study_name, storage=storage_name)
    
    # Loading best trial parameters
    C = loaded_study.best_trials[0].params['C']
    degree = loaded_study.best_trials[0].params['degree']
    kernel= loaded_study.best_trials[0].params['kernel']

    # Reinitialize the model
    params = {
        'C':C,
        'degree':degree,

        'kernel':kernel
        }
    model = SVC()  
    model.set_params(**params)
    
    # Retrain the best model
    train_set_data_tr, test_set_data_tr, train_set_labels_np, test_set_labels_np, pipe = dataset_creation(df,name)
    model.fit(train_set_data_tr, train_set_labels_np)
    y_pred = model.predict(test_set_data_tr)
    response_accuracy[f'SVC_{i+1}'] = round(model.score(test_set_data_tr, test_set_labels_np),3)
    # Create the Confusion Matrix for each model
    disp = metrics.ConfusionMatrixDisplay.from_predictions(test_set_labels_np, y_pred)
    disp.figure_.suptitle(f"SVC Confusion Matrix for the {i+1} combination")
    plt.savefig(f"Confusion_Matrices//SVC Confusion Matrix for the {i+1} combination.jpg")

<h2><center>MPL</center></h2>

In [ ]:
for i, name in enumerate(input_list):
    print(i)
    comb = i+1
    # Loading the study
    study_name = f"MLP_study_{i}"  
    storage_name = f"sqlite:///{study_name}.db"
    loaded_study = optuna.load_study(study_name=study_name, storage=storage_name)
    
    # Loading best trial parameters and initializing the model
    train_set_data_tr, test_set_data_tr, train_set_labels_np, test_set_labels_np, pipe = dataset_creation(df,name)
    num_layers = loaded_study.best_trials[0].params['num_layers']
    model = tf.keras.Sequential()
    model.add(Input(train_set_data_tr.shape[1]))
    model.add(Dense(5*train_set_data_tr.shape[1],activation='relu',kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0)))
    for i in range(num_layers):
        neurons = loaded_study.best_trials[0].params[f"neurons_{i}"]
        model.add(Dense(neurons, activation="relu",kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0)))
    model.add(Dense(80,activation='relu',kernel_initializer="he_normal",kernel_regularizer=tf.keras.regularizers.l2(0.0)))
    model.add(Dense(4, activation='softmax'))
    kwargs = {}
    kwargs["learning_rate"] = loaded_study.best_trials[0].params["adam_learning_rate"]
    optimizer = getattr(tf.optimizers, "Adam")(**kwargs)
    epochs = loaded_study.best_trials[0].params["epochs"]
    batch_size = loaded_study.best_trials[0].params["batch_size"]
    model.compile(loss='sparse_categorical_crossentropy', optimizer=optimizer, metrics=['accuracy'])

    
    # Retrain the best model

    model.fit( tf.convert_to_tensor(train_set_data_tr), train_set_labels_np, epochs=epochs, batch_size=300)
    response_accuracy[f'MLP_{i}'] = round(model.evaluate(test_set_data_tr, test_set_labels_np)[1],3)
    y_pred = np.argmax(model.predict(test_set_data_tr),axis=1)
    # Create the Confusion Matrix for each model
    disp = metrics.ConfusionMatrixDisplay.from_predictions(test_set_labels_np, y_pred)
    disp.figure_.suptitle(f"MPL Confusion Matrix for the {comb} combination")
    plt.savefig(f"Confusion_Matrices//MPL Confusion Matrix for the {comb} combination.jpg")

In [ ]:
accuracy_sorted_keys_opt = sorted(response_accuracy, key=response_accuracy.get)  # [1, 3, 2]
accuracy_sorted_dict_opt= {}
for w in accuracy_sorted_keys_opt:
    accuracy_sorted_dict_opt[w] = response_accuracy[w]

names = list(accuracy_sorted_dict_opt.keys())
accuracy_values = list(accuracy_sorted_dict_opt.values())


x = np.arange(len(names))  # the label locations
width = 0.35  # the width of the bars

fig, ax = plt.subplots(figsize = (15,7))
rects1 = ax.bar(x - width/2, accuracy_values, width, label='accuracy_score')
ax.set_ylabel('Scores')
ax.set_title('Trainining Scores accuracy')
ax.set_xticks(x, names, rotation = 90)
ax.legend()

ax.bar_label(rects1, padding=3)

plt.savefig(f"Confusion_Matrices//Accuracies.jpg")

<h2><center>Retrive the best parameters per input combination</center></h2>

In [ ]:

for i in range(4):
    print(i)
    study_name = f"MLP_study_{i}"  
    storage_name = f"sqlite:///{study_name}.db"
    loaded_study = optuna.load_study(study_name=study_name, storage=storage_name)
    print(loaded_study.best_trials[0].params)

In [ ]:
for i in range(4):
    print(i)
    study_name = f"studies//SVC_study_{i}"  
    storage_name = f"sqlite:///{study_name}.db"
    loaded_study = optuna.load_study(study_name=study_name, storage=storage_name)
    print(loaded_study.best_trials[0].params)